In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.contrib import rnn
from tensorflow.contrib.crf import crf_log_likelihood, viterbi_decode    # 条件随机场的对数似然，维特比解码

In [ ]:
def network(inputs, shapes, num_tags, lstm_dim = 100, initializer = tf.truncated_normal_initializer()):
    """
    接收一个批次样本的特征数据，计算出网络的输出值
    :param char:type of int,id of chars  a tensor of shape 2-D [None, None]
    bound, flag, radical, pinyin同上
    """
    # - - - - - - - - - - - - - -特征嵌入- - - - - - - - - - - - - - - -
    #将所有特征的id转换成一个固定长度的向量
    embedding = []
    keys = list(shapes.keys())
    for key in keys:
        # tf.name_scope()、tf.variable_scope()是两个作用域函数，主要用于变量共享
            lookup = tf.get_variable(
                name = key + '_embedding',
                shape = shapes[key],
                initializer = initializer
            )
            embedding.append(tf.nn.embedding_lookup(lookup, inputs[key]))
    embed = tf.concat(embedding, axis = -1)
    
    sign = tf.sign(tf.abs(inputs[keys[0]]))
    lengths = tf.reduce_sum(sign, reduction_indices = 1)
    
    # - - - - - - - - - - - - - -循环神经网络编码- - - - - - - - - - - - - - - -
    with tf.variable_scope('BiLstm_layer1'):
        lstm_cell = {}
        for name in ['forward', 'backward']:
            with tf.variable_scope(name):
                lstm_cell[name] = rnn.BasicLSTMCell(
                    lstm_dim
                )
        outputs1, final_states1 = tf.nn.bidirectional_dynamic_rnn(
            lstm_cell['forward'],
            lstm_cell['backward'],
            embed,
            dtype = tf.float32,
            sequence_length = lengths
        )
    output1 = tf.concat(outputs1, axis = -1)
    
    with tf.variable_scope('BiLstm_layer2'):
        lstm_cell = {}
        for name in ['forward', 'backward']:
            with tf.variable_scope(name):
                lstm_cell[name] = rnn.BasicLSTMCell(
                    lstm_dim
                )
        outputs, final_states = tf.nn.bidirectional_dynamic_rnn(
                # final_states是最终状态，包含前向和反向的c和h（c: 长时记忆，h: 短时记忆）
            lstm_cell['forward'],
            lstm_cell['backward'],
            output1,
            dtype = tf.float32,
            sequence_length = lengths    # 每句话的实际长度
        )
    output = tf.concat(outputs, axis = -1)
    num_time = tf.shape(inputs[keys[0]])[1]
    
    
    # - - - - - - - - - - - - - -输出映射- - - - - - - - - - - - - - - -
    output = tf.reshape(output, [-1, 2 * lstm_dim])
    with tf.variable_scope('project_layer1'):
        w = tf.get_variable(
            name = 'w',
            shape = [2 * lstm_dim, lstm_dim],
            initializer = initializer
        )
        b = tf.get_variable(
            name = 'b',
            shape = [lstm_dim],
            initializer = tf.zeros_initializer()
        )
        output = tf.nn.relu(tf.matmul(output, w) + b)
        
    with tf.variable_scope('project_layer2'):
        w = tf.get_variable(
            name = 'w',
            shape = [lstm_dim, num_tags],
            initializer = initializer
        )
        b = tf.get_variable(
            name = 'b',
            shape = [num_tags],
            initializer = tf.zeros_initializer()
        )
        output = tf.matmul(output, w) + b
    output = tf.reshape(output, [-1, num_time, num_tags])
    
    return output, lengths    # batch_size, max_length, num_tags
    
class Model(object):
    
    def __init__(self, dict, lr = 0.0001):
        # - - - - -  -用到的参数值-- - - - - - - -
        self.num_char = len(dict['word'][0])
        self.num_bound = len(dict['bound'][0])
        self.num_flag = len(dict['flag'][0])
        self.num_radical = len(dict['radical'][0])
        self.num_pinyin = len(dict['pinyin'][0])
        self.num_tags = len(dict['label'][0])
        self.char_dim = 100
        self.bound_dim = 20
        self.flag_dim = 50
        self.radical_dim = 50
        self.pinyin_dim = 50
        self.lstm_dim = 100    # lstm的神经元个数
        self.lr = lr    # 学习率
        self.map = dict
        
        # - - - - - - - - - - - - - - - -定义接收数据的placeholder- - - - - - - - - - - - - - - - - - 
        # tf.placeholder(dtype, shape=None, name=None): tensorFlow中的占位符，用于传入外部数据
        self.char_inputs = tf.placeholder(dtype = tf.int32, shape = [None, None], name = 'char_inputs')
        self.bound_inputs = tf.placeholder(dtype = tf.int32, shape = [None, None], name = 'bound_inputs')
        self.flag_inputs = tf.placeholder(dtype = tf.int32, shape = [None, None], name = 'flag_inputs')
        self.radical_inputs = tf.placeholder(dtype = tf.int32, shape = [None, None], name = 'radical_inputs')
        self.pinyin_inputs = tf.placeholder(dtype = tf.int32, shape = [None, None], name = 'pinyin_inputs')
        self.targets = tf.placeholder(dtype = tf.int32, shape = [None, None], name = 'targets')    # 真实值
        self.global_step = tf.Variable(0, trainable = False)
        
        # - - - - - - - - - - - -计算模型输出值-- - - - - - - - - - - - - 
        self.logits, self.lengths = self.get_logits(self.char_inputs,
                                                    self.bound_inputs,
                                                    self.flag_inputs,
                                                    self.radical_inputs,
                                                    self.pinyin_inputs
                                                    )
        # - - - - - - - - - - - - - - - - -计算损失- - - - - - - - - - - - - - - - - - 
        self.cost = self.loss(self.logits, self.targets, self.lengths)
        
        # - - - - - - - - - - - -优化器优化-- - - - - - - - - - - - - 
        # 采用梯度截断技术
        with tf.variable_scope('optimizer'):
            opt = tf.train.AdamOptimizer(self.lr)
            grad_vars = opt.compute_gradients(self.cost)
            clip_grad_vars = [[tf.clip_by_value(g, -5, 5), v] for g, v in grad_vars]
                # tf.clip_by_value():可以将一个张量中的数值限制在一个范围之内
            self.train_op = opt.apply_gradients(clip_grad_vars, self.global_step)
        self.saver = tf.train.Saver(tf.global_variables(), max_to_keep = 5)
        
    
    def get_logits(self, char, bound, flag, radical, pinyin):    # 获取模型的输出值
        """
        :param char:type of int,id of chars  a tensor of shape 2-D [None, None]
        bound, flag, radical, pinyin同上
        :return:3-d tensor [batch_size, max_length, num_tags]
        """
        shapes = {}
        shapes['char'] = [self.num_char, self.char_dim]
        shapes['bound'] = [self.num_bound, self.bound_dim]
        shapes['flag'] = [self.num_flag, self.flag_dim]
        shapes['radical'] = [self.num_radical, self.radical_dim]
        shapes['pinyin'] = [self.num_pinyin, self.pinyin_dim]
        inputs = {}
        inputs['char'] = char
        inputs['bound'] = bound
        inputs['flag'] = flag
        inputs['radical'] = radical
        inputs['pinyin'] = pinyin
        
        return network(inputs, shapes, lstm_dim = self.lstm_dim, num_tags = self.num_tags)
    
    def loss(self, output, targets, lengths):
        batch_size = tf.shape(lengths)[0]
        num_steps = tf.shape(output)[1]
        with tf.variable_scope('crf_loss'):
            small = -1000.0
            start_logits = tf.concat(
                [small * tf.ones(shape = [batch_size, 1, self.num_tags]), tf.zeros(shape = [batch_size, 1, 1])], axis = -1
            )
            pad_logits = tf.cast(small * tf.ones([batch_size, num_steps, 1]), tf.float32)
            logits = tf.concat([output, pad_logits], axis = -1)
            logits = tf.concat([start_logits, logits], axis = 1)
            targets = tf.concat(
                [tf.cast(self.num_tags * tf.ones([batch_size, 1]), tf.int32), targets], axis = -1
            )
            self.trans = tf.get_variable(
                name = 'trans',
                shape = [self.num_tags + 1, self.num_tags + 1],
                initializer = tf.truncated_normal_initializer()
            )
            log_likelihood, self.trans = crf_log_likelihood(
                inputs = logits, 
                tag_indices = targets,
                transition_params = self.trans,
                sequence_lengths = lengths
            )
            return tf.reduce_mean(-log_likelihood)
        
    def run_step(self, sess, batch, istrain = True):
        if istrain:
            feed_dict = {
                self.char_inputs: batch[0],
                self.bound_inputs: batch[1],
                self.flag_inputs: batch[2],
                self.radical_inputs: batch[3],
                self.pinyin_inputs: batch[4],
                self.targets: batch[5]
            }
            _, loss = sess.run([self.train_op, self.cost], feed_dict = feed_dict)
            return loss
        else:
            feed_dict = {
                self.char_inputs: batch[0],
                self.bound_inputs: batch[1],
                self.flag_inputs: batch[2],
                self.radical_inputs: batch[3],
                self.pinyin_inputs: batch[4]
            }
            logits, lengths = sess.run([self.logits, self.lengths], feed_dict = feed_dict)
            return logits, lengths
        
    def decode(self, logits, lengths, matrix):
        paths = []
        small = -1000.0
        start = np.asarray([[small] * self.num_tags + [0]])
        for score, length in zip(logits, lengths):
            score = score[:length]
            pad = small * np.ones([length, 1])
            logit = np.concatenate([score, pad], axis = -1)
            logit = np.concatenate([start, logit], axis = 0)
            path, _ = viterbi_decode(logit, matrix)
            
            paths.append(path[1:])
        return paths
    
    def predict(self, sess, batch):
        results = []
        matrix = self.trans.eval()    # eval()函数用来执行一个字符串表达式，并返回表达式的值
        logits, lengths = self.run_step(sess, batch, istrain = False)
        paths = self.decode(logits. lengths, matrix)
        chars = batch[0]
        for i in range(len(paths)):
            length = lengths[i]
            string = [self.map['word'][0][index] for index in chars[i][:length]]
            tags = [self.map['label'][0][index] for index in paths[i]]
            result = [k for k in zip(string, tags)]
            results.append(result)
        return results

In [ ]:
# matrix
# x
result = tf.nn.embedding_lookup(matrix, x)
tf.compat.v1.disable_eager_execution()
sess = tf.compat.v1.Session()
print(sess.run(result))